## 问题定义

本问题为求解2D Helmholtz方程。其形式可以参考案例18中的PDE定义。


### 求解目标

要求求解给定区域内的Helmholtz方程

## HardBC

参考论文：Sukumar, N., and Ankit Srivastava. “Exact imposition of boundary conditions with distance functions in physics-informed deep neural networks.” Computer Methods in Applied Mechanics and Engineering 389 (2022): 114333.

在一般的训练流程中，所有的定解条件均是以惩罚项的形式添加到损失函数中，也就是软约束。这在一些问题中会影响求解，因为这种情况下无法保证精确满足定解条件。

这篇文章从模型定义出发，将BC条件编入了模型中，从而保证BC的严格成立。


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter

In [3]:
import torch
import numpy as np
from sympy import Symbol, pi, sin
from typing import List, Tuple, Dict

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.geometry.adf import ADF
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 20000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: false
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch

### 定义必要组件

#### 计算ADF

In [5]:
class HardBC(ADF):  # 计算ADF
    def __init__(self):
        super().__init__()

        # domain measures
        # 几何
        self.domain_height: float = 2.0
        self.domain_width: float = 2.0

        # boundary conditions (bottom, right, top, left)
        # 边界条件
        # 本问题是比较简单的Dirichlet BC
        self.g: List[float] = [0.0, 0.0, 0.0, 0.0]

        # parameters
        self.eps: float = 1e-9
        self.mu: float = 2.0
        self.m: float = 2.0

    def forward(self, invar: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Forms the solution anstaz for the Helmholtz example
        """

        outvar = {}
        x, y = invar["x"], invar["y"]
        
        # 四条边的分量
        omega_0 = ADF.line_segment_adf(
            (x, y),
            (-self.domain_width / 2, -self.domain_height / 2),
            (self.domain_width / 2, -self.domain_height / 2),
        )
        omega_1 = ADF.line_segment_adf(
            (x, y),
            (self.domain_width / 2, -self.domain_height / 2),
            (self.domain_width / 2, self.domain_height / 2),
        )
        omega_2 = ADF.line_segment_adf(
            (x, y),
            (self.domain_width / 2, self.domain_height / 2),
            (-self.domain_width / 2, self.domain_height / 2),
        )
        omega_3 = ADF.line_segment_adf(
            (x, y),
            (-self.domain_width / 2, self.domain_height / 2),
            (-self.domain_width / 2, -self.domain_height / 2),
        )
        
        # 计算phi
        omega_E_u = ADF.r_equivalence([omega_0, omega_1, omega_2, omega_3], self.m)

        bases = [
            omega_0**self.mu,
            omega_1**self.mu,
            omega_2**self.mu,
            omega_3**self.mu,
        ]
        w = [
            ADF.transfinite_interpolation(bases, idx, self.eps)
            for idx in range(len(self.g))
        ]
        g = w[0] * self.g[0] + w[1] * self.g[1] + w[2] * self.g[2] + w[3] * self.g[3]
        
        # 模型部分
        outvar["u"] = g + omega_E_u * invar["u_star"]
        return outvar

In [6]:
hard_bc = HardBC()

#### Geo

In [7]:
x, y = Symbol("x"), Symbol("y")
height = hard_bc.domain_height
width = hard_bc.domain_width
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### PDE

In [8]:
wave = HelmholtzEquation(u="u", k=1.0, dim=2, mixed_form=True)  # 亥姆霍兹方程，注意这里需要定义mixed_form=True，从而启用first order method

#### Model

In [9]:
# 定义简单的全连接网络
# 输入为空间坐标
# 输出为模型预测结果以及对应的一阶项（关于此，请参考官方文档中的first order method）
# https://docs.nvidia.com/deeplearning/modulus/modulus-sym/user_guide/theory/advanced_schemes.html#exact-boundary-condition-imposition
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u_star"), Key("u_x"), Key("u_y")],
    cfg=cfg.arch.fully_connected,
)

#### Node

In [10]:
nodes = (
    wave.make_nodes()  # PDE
    + [Node(inputs=["x", "y", "u_star"], outputs=["u"], evaluate=hard_bc)]  # HardBC
    + [wave_net.make_node(name="wave_network")]  # model
)

#### Domain

In [11]:
domain = Domain()

#### ABC边界

由于HardBC硬编码，所以不需要引入边界条件

#### PDE约束

In [12]:
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "helmholtz": -(
            -((pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            - ((4 * pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            + 1 * sin(pi * x) * sin(4 * pi * y)
        ),  # PDE约束
        "compatibility_u_x": 0,  # 一阶项预测约束
        "compatibility_u_y": 0,  # 一阶项预测约束
    },
    batch_size=cfg.batch_size.interior,
    bounds={x: (-width / 2, width / 2), y: (-height / 2, height / 2)},
    lambda_weighting={
        "helmholtz": Symbol("sdf"),
        "compatibility_u_x": 0.5,
        "compatibility_u_y": 0.5,
    },
)
domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [13]:
file_path = "openfoam/helmholtz.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x": "x", "y": "y", "z": "u"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u"]
    }

    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [14]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [15]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [16]:
slv.solve()

2024-03-03 08:04:03,313 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-03 08:04:03,320 - INFO - attempting to restore from: /workspace/22_2D_Helmholtz_HardBC/outputs
2024-03-03 08:04:03,335 - INFO - Success loading optimizer: /workspace/22_2D_Helmholtz_HardBC/outputs/optim_checkpoint.0.pth
2024-03-03 08:04:03,342 - INFO - Success loading model: /workspace/22_2D_Helmholtz_HardBC/outputs/wave_network.0.pth
2024-03-03 08:04:04,269 - INFO - [step:          0] record constraint batch time:  4.433e-02s
2024-03-03 08:04:15,591 - INFO - [step:          0] record validators time:  1.132e+01s
2024-03-03 08:04:15,692 - INFO - [step:          0] saved checkpoint to /workspace/22_2D_Helmholtz_HardBC/outputs
2024-03-03 08:04:15,695 - INFO - [step:          0] loss:  1.002e+04
2024-03-03 08:04:25,220 - INFO - [step:        100] loss:  9.968e+03, time/iteration:  9.524e+01 ms
2024-03-03 08:04:34,22

2024-03-03 08:14:43,256 - INFO - [step:       6000] loss:  6.231e-01, time/iteration:  2.098e+02 ms
2024-03-03 08:14:52,748 - INFO - [step:       6100] loss:  2.392e-01, time/iteration:  9.491e+01 ms
2024-03-03 08:15:01,492 - INFO - [step:       6200] loss:  1.258e-01, time/iteration:  8.742e+01 ms
2024-03-03 08:15:10,968 - INFO - [step:       6300] loss:  6.574e-01, time/iteration:  9.473e+01 ms
2024-03-03 08:15:20,802 - INFO - [step:       6400] loss:  4.958e-01, time/iteration:  9.832e+01 ms
2024-03-03 08:15:30,172 - INFO - [step:       6500] loss:  1.138e-01, time/iteration:  9.368e+01 ms
2024-03-03 08:15:38,398 - INFO - [step:       6600] loss:  2.576e-01, time/iteration:  8.224e+01 ms
2024-03-03 08:15:47,070 - INFO - [step:       6700] loss:  1.390e-01, time/iteration:  8.669e+01 ms
2024-03-03 08:15:56,589 - INFO - [step:       6800] loss:  2.269e-01, time/iteration:  9.516e+01 ms
2024-03-03 08:16:06,210 - INFO - [step:       6900] loss:  5.130e-01, time/iteration:  9.619e+01 ms


2024-03-03 08:26:21,668 - INFO - [step:      12700] loss:  3.738e-02, time/iteration:  9.904e+01 ms
2024-03-03 08:26:30,305 - INFO - [step:      12800] loss:  4.127e-02, time/iteration:  8.635e+01 ms
2024-03-03 08:26:39,415 - INFO - [step:      12900] loss:  3.706e-02, time/iteration:  9.108e+01 ms
2024-03-03 08:27:00,985 - INFO - [step:      13000] record validators time:  1.159e+01s
2024-03-03 08:27:01,089 - INFO - [step:      13000] saved checkpoint to /workspace/22_2D_Helmholtz_HardBC/outputs
2024-03-03 08:27:01,092 - INFO - [step:      13000] loss:  3.540e-02, time/iteration:  2.168e+02 ms
2024-03-03 08:27:10,330 - INFO - [step:      13100] loss:  8.292e-02, time/iteration:  9.237e+01 ms
2024-03-03 08:27:19,453 - INFO - [step:      13200] loss:  3.396e-02, time/iteration:  9.121e+01 ms
2024-03-03 08:27:28,802 - INFO - [step:      13300] loss:  3.243e-02, time/iteration:  9.348e+01 ms
2024-03-03 08:27:38,253 - INFO - [step:      13400] loss:  4.114e-02, time/iteration:  9.450e+01 m

2024-03-03 08:37:15,513 - INFO - [step:      19200] loss:  2.738e-02, time/iteration:  8.173e+01 ms
2024-03-03 08:37:22,775 - INFO - [step:      19300] loss:  2.526e-02, time/iteration:  7.260e+01 ms
2024-03-03 08:37:32,541 - INFO - [step:      19400] loss:  2.766e-02, time/iteration:  9.764e+01 ms
2024-03-03 08:37:42,096 - INFO - [step:      19500] loss:  2.848e-02, time/iteration:  9.552e+01 ms
2024-03-03 08:37:50,737 - INFO - [step:      19600] loss:  2.475e-02, time/iteration:  8.639e+01 ms
2024-03-03 08:38:00,143 - INFO - [step:      19700] loss:  2.433e-02, time/iteration:  9.403e+01 ms
2024-03-03 08:38:08,925 - INFO - [step:      19800] loss:  2.437e-02, time/iteration:  8.780e+01 ms
2024-03-03 08:38:18,167 - INFO - [step:      19900] loss:  2.325e-02, time/iteration:  9.240e+01 ms
2024-03-03 08:38:27,347 - INFO - [step:      20000] record constraint batch time:  8.903e-02s
2024-03-03 08:38:40,349 - INFO - [step:      20000] record validators time:  1.300e+01s
2024-03-03 08:38:4

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)